In [1]:
import sys
import os
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from harp.clock import decode_harp_clock, align_timestamps_to_anchor_points
from open_ephys.analysis import Session
import datetime
from aind_ephys_rig_qc.temporal_alignment import search_harp_line
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from scipy.io import loadmat
from scipy.stats import zscore
import ast
from utils.plot_utils import combine_pdf_big

from open_ephys.analysis import Session
from pathlib import Path
import glob

import json
import seaborn as sns
from PyPDF2 import PdfMerger
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import re
from aind_dynamic_foraging_basic_analysis.plot.plot_foraging_session import plot_foraging_session
from aind_dynamic_foraging_data_utils.nwb_utils import load_nwb_from_filename
from hdmf_zarr.nwb import NWBZarrIO
from utils.beh_functions import session_dirs, parseSessionID, load_model_dv, makeSessionDF, get_session_tbl, get_unit_tbl, get_history_from_nwb
from utils.ephys_functions import*
from utils.opto_utils import opto_metrics, load_opto_sig
import pandas as pd
import pickle
import scipy.stats as stats
from joblib import Parallel, delayed
from multiprocessing import Pool
from functools import partial
import time
import spikeinterface as si
import shutil 
import seaborn as sns
import math
import seaborn as sns
from sklearn.decomposition import PCA
from scipy.stats import zscore
%matplotlib inline

In [2]:
# Make combined session-unit table
dfs = [pd.read_csv('/root/capsule/code/data_management/session_assets.csv'),
        pd.read_csv('/root/capsule/code/data_management/hopkins_session_assets.csv')]
df = pd.concat(dfs)
# df = df[-10:]
session_ids = df['session_id'].values
behs = df['behavior'].values
exclude = ['ecephys_717120_2024-03-06_12-23-53', 'ecephys_713854_2024-03-08_14-54-25', 'ecephys_713854_2024-03-08_16-20-33', 'behavior_754897_2025-03-15_11-32-18']
session_ids, behs = zip(*[
    (session, beh)
    for session, beh in zip(session_ids, behs)
    if isinstance(session, str) and session not in exclude
])
session_ids = list(session_ids)
behs = list(behs)

In [3]:
# loop through all sessions
# count number of units pass qc and opto qc
# check if exist in longer than 100 sessions 
# check if exist in pre/post stimulation
# check if exist in anti-dromic stimulation
all_sessions = []
all_units = []
all_qc_pass_units = [] # default qc pass
all_real_units = [] # not noise, not artifact
all_tagged_units = []
in_behavior = []
trial_count = []
opto_tagging_pre = []
opto_tagging_post = []
anti = []
probes = []
all_p_max = []
all_p_mean = []
all_lat_max_p = []  # latency of max p response
isi_v = []
all_eu = []
all_corr = []
all_amp = []
all_peak = []
all_wf = []
all_wf_aligned = []
all_wf_raw = []
all_wf_2d = []
all_wf_2d_raw = []
all_peak_raw = []
all_amp_raw = []
y_loc = []
in_df = []
rec_side = []
top = []
bottom = []
snr = []
all_tag_loc = []
all_fr = []
all_sig_counts = []
all_decoder = []
all_peak_channel_ind = []
resp_p_all_conditions = []
mean_p_all_conditions = []
resp_lat_all_conditions = []
corr_all_conditions = []
eu_all_conditions = []
sig_counts_all_conditions = []
all_x = []
all_y = []
all_z = []

# p_resp_thresh = 0.5
# lat_resp_thresh = 0.02


In [5]:
target = 'soma'
for session, beh in zip(session_ids, behs):
    session_dir = session_dirs(session)
    if 'ZS' in session:
        if (not os.path.exists(session_dir['nwb_dir_raw'])) or (get_unit_tbl(session, 'curated') is None):
            print(f'Skipping {session} due to no neuron data')
            continue
    if session_dir['curated_dir_curated'] is not None:
        # if 'behavior_761038_2025-04-15_10-25-11' in session:
        #     print('behavior_761038_2025-04-15_10-25-11')
        print(f'Processing {session}')
        data_type = 'curated'
        # if session_dir['curated_dir_curated'] is not None:
        #     data_type = 'curated'
        # elif session_dir['curated_dir_raw'] is not None:
        #     data_type = 'raw'
        # else:
        #     continue
        qm_file = os.path.join(session_dir['processed_dir'], )
        unit_tbl = get_unit_tbl(session, data_type)
        opto_metrics_session = opto_metrics(session, data_type=data_type)
        # unit_nwb = load_nwb_from_filename(session_dir['nwb_dir_curated'])
        # unit_temps = unit_nwb.units[:]['waveform_mean'].values
        # peak_C = [np.argmax(np.ptp(curr_wf, axis=0)) if isinstance(curr_wf, np.ndarray) else None for curr_wf in unit_temps]  # peak channel index
        # print(np.shape(unit_temps[0]))
        all_units.extend(unit_tbl['unit_id'].tolist())
        all_qc_pass_units.extend(unit_tbl['default_qc'].tolist())  # default qc pass
        # all_real_units.extend(unit_tbl['real_unit'].tolist())
        all_tagged_units.extend(unit_tbl['tagged_loc'].tolist())  # tagged location (e.g. 'soma', 'axon', 'unspecified')
        all_sessions.extend([session]*len(unit_tbl))
        # all_peak_channel_ind.extend(peak_C)  # peak channel index
        if 'p_max' not in unit_tbl.columns:
            all_p_max.extend(unit_tbl['p_max_x'].tolist())
            all_p_mean.extend(unit_tbl['p_mean_x'].tolist())
            all_lat_max_p.extend(unit_tbl['lat_max_p_x'].tolist())
            all_eu.extend(unit_tbl['euc_max_p_x'].tolist())
            all_corr.extend(unit_tbl['corr_max_p_x'].tolist())
            peaks = unit_tbl['peak_x'].values
            amp = unit_tbl['amp_x'].values
        else: 
            all_p_max.extend(unit_tbl['p_max'].tolist())
            all_p_mean.extend(unit_tbl['p_mean'].tolist())  
            all_lat_max_p.extend(unit_tbl['lat_max_p'].tolist()) 
            all_eu.extend(unit_tbl['euc_max_p'].tolist())
            all_corr.extend(unit_tbl['corr_max_p'].tolist()) 
            peaks = unit_tbl['peak'].values
            amp = unit_tbl['amp'].values
        
        if 'x_ccf' in unit_tbl.columns:
            all_x.extend(unit_tbl['x_ccf'].tolist())
            all_y.extend(unit_tbl['y_ccf'].tolist())
            all_z.extend(unit_tbl['z_ccf'].tolist())
        else:
            all_x.extend([np.nan]*len(unit_tbl))
            all_y.extend([np.nan]*len(unit_tbl))
            all_z.extend([np.nan]*len(unit_tbl))
     
        isi_v.extend(unit_tbl['isi_violations_ratio'].tolist())  # ISI violations 
        snr.extend(unit_tbl['snr'].tolist())  # signal-to-noise ratio
        y_loc.extend(unit_tbl['y_loc'].tolist())  # y location of the unit
        all_fr.extend(unit_tbl['firing_rate'].tolist())  # firing rate
        all_decoder.extend(unit_tbl['decoder_label'].tolist())  # decoder value
        if 'tagged_loc' in unit_tbl.columns:
            all_tag_loc.extend(unit_tbl['tagged_loc'].tolist())
        else:
            all_tag_loc.extend([np.nan]*len(unit_tbl))
        if 'peak_wf_opt' in unit_tbl.columns:
            wf_opt = [wf_opt_unit if isinstance(wf_opt_unit, np.ndarray) else wf_unit for wf_opt_unit, wf_unit in zip(unit_tbl['peak_wf_opt'], unit_tbl['peak_wf'])]  # peak waveform
            wf_opt_aligned = [wf_opt_unit if isinstance(wf_opt_unit, np.ndarray) else wf_unit for wf_opt_unit, wf_unit in zip(unit_tbl['peak_wf_opt_aligned'], unit_tbl['peak_wf_aligned'])]  # peak waveform aligned
            wf_opt_2d = [wf_opt_unit if isinstance(wf_opt_unit, np.ndarray) else wf_unit for wf_opt_unit, wf_unit in zip(unit_tbl['mat_wf_opt'], unit_tbl['wf_2d'])]  # peak waveform 2D
        else:
            wf_opt = unit_tbl['peak_wf'].values.tolist()
            wf_opt_aligned = unit_tbl['peak_wf_aligned'].values.tolist()
            wf_opt_2d = unit_tbl['wf_2d'].values.tolist()

        amp_opt = [
                        np.max(wf_opt_curr) - np.min(wf_opt_curr) if isinstance(wf_opt_curr, np.ndarray) else curr_amp_unit
                        for wf_opt_curr, curr_amp_unit in zip(wf_opt, amp)
                    ]   # amplitude of optimized waveforms
        if 'amplitude_opt' in unit_tbl.columns:
            peak_opt = [
                            curr_peak_opt_unit if not np.isnan(curr_peak_opt_unit) else curr_peak_unit
                            for curr_peak_opt_unit, curr_peak_unit in zip(list(unit_tbl['amplitude_opt'].values), list(peaks))
                        ]
        else:
            peak_opt = list(peaks)
        
        if 'peak_waveform_raw_aligned' in unit_tbl.columns:
            wf_raw = unit_tbl['peak_waveform_raw_fake_aligned'].values.tolist()
            wf_2d_raw = unit_tbl['mat_wf_raw_fake'].values.tolist()
            peak_raw = unit_tbl['peak_raw_fake'].values.tolist()
            peak_raw = [curr_peak_raw-curr_wf[0] if (curr_peak_raw is not None) and (~np.isnan(curr_peak_raw)) else None for curr_peak_raw, curr_wf in zip(peak_raw, wf_raw)]
            amp_raw = unit_tbl['amplitude_raw_fake'].values.tolist()
        else:
            wf_raw = [None]*len(unit_tbl)
            wf_2d_raw = [None]*len(unit_tbl)
            peak_raw = [None]*len(unit_tbl)
            amp_raw = [None]*len(unit_tbl)
        
        all_amp.extend(amp_opt)  # amplitude 
        all_peak.extend(peak_opt)  # peak opto response
        all_wf.extend(wf_opt)  # waveform 
        all_wf_aligned.extend(wf_opt_aligned)  # waveform of max amp
        all_wf_2d.extend(wf_opt_2d)  # waveform of max amp
        all_wf_raw.extend(wf_raw)  # raw waveform
        all_wf_2d_raw.extend(wf_2d_raw)  # raw waveform 2d
        all_peak_raw.extend(peak_raw)  # raw peak
        all_amp_raw.extend(amp_raw)  # raw amplitude
        
        top.extend(unit_tbl['LC_range_top'].tolist())
        bottom.extend(unit_tbl['LC_range_bottom'].tolist())

        # session and opto information
        session_df = get_session_tbl(session)
        session_opto_sig = load_opto_sig(session, data_type=data_type)
        # with open(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_info_{target}.json')) as f:
        #     opto_info = json.load(f)

        if not session_dir['aniID'].startswith('ZS'):
            opto_df = pd.read_csv(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_session_{target}.csv'))
            if len(opto_df[opto_df['pre_post'] == 'pre'])>0:
                pre_end = np.max(opto_df[opto_df['pre_post'] == 'pre']['time'].values)
            else:
                pre_end = np.nan
        
            if len(opto_df[opto_df['pre_post'] == 'post'])>0:
                post_start = np.min(opto_df[opto_df['pre_post'] == 'post']['time'].values)
                post_end = np.max(opto_df[opto_df['pre_post'] == 'post']['time'].values)
            else:
                post_start = np.nan
                post_end = np.nan
        
        for unit_id in unit_tbl['unit_id'].values:
            # append opto_metrics
            spike_times = unit_tbl[unit_tbl['unit_id'] == unit_id]['spike_times'].values[0]
            unit_opto = opto_metrics_session.load_unit(unit_id)
            unit_opto_sig = session_opto_sig.load_unit(unit_id)

            if unit_opto_sig is not None:
                curr_max_count = unit_opto_sig['p_sig_count'].max()
            else:
                curr_max_count = np.nan
            
            curr_p_resp_all = unit_opto['resp_p_bl'].values
            curr_lat_resp_all = unit_opto['resp_lat'].values
            curr_p_mean_all = unit_opto['mean_p'].values
            curr_eu_all = unit_opto['euclidean_norm'].values
            curr_corr_all = unit_opto['correlation'].values
            unit_opto['sig_num'] = np.full(len(unit_opto), np.nan)
            if unit_opto_sig is not None:
                if not session_dir['aniID'].startswith('ZS'):
                    for cond_ind, row in unit_opto.iterrows():
                        filter = (unit_opto_sig['power'] == row['powers']) & (unit_opto_sig['site'] == row['sites'])
                        if len(unit_opto_sig['pre_post'].unique()) > 1:
                            filter &= (unit_opto_sig['pre_post'] == row['stim_times'])
                        curr_sig_rows = unit_opto_sig[filter]
                        if len(curr_sig_rows) == 1:
                            unit_opto.loc[cond_ind, 'sig_num'] = curr_sig_rows['p_sig_count'].values[0]
                        elif len(curr_sig_rows) > 1:
                            print(f'Warning: multiple opto sig rows for unit {unit_id} in session {session} with condition {row["powers"]} and pre_post {row["stim_times"]}')
                            unit_opto.loc[cond_ind, 'sig_num'] = curr_sig_rows['p_sig_count'].values[0]
                        else:
                            print(f'Warning: no opto sig rows for unit {unit_id} in session {session} with condition {row["powers"]} and pre_post {row["stim_times"]}')
                            unit_opto.loc[cond_ind, 'sig_num'] = np.nan
                else:
                    unit_opto['sig_num'] = unit_opto_sig['p_sig_count'].values[0]
            
            curr_sig_num_all = unit_opto['sig_num'].values

            resp_p_all_conditions.append(curr_p_resp_all)
            resp_lat_all_conditions.append(curr_lat_resp_all)
            corr_all_conditions.append(curr_corr_all)
            eu_all_conditions.append(curr_eu_all)
            mean_p_all_conditions.append(curr_p_mean_all)
            sig_counts_all_conditions.append(curr_sig_num_all)
            all_sig_counts.append(curr_max_count)


            if session_dir['aniID'].startswith('ZS'):
                curr_anti_opto = False
                curr_pre_opto = False
                curr_post_opto = True
            else:
                curr_anti_opto = True
                curr_pre_opto = True
                curr_post_opto = True  
            if session_df is not None:
                curr_in_beh = True
            else:
                curr_in_beh = False
                curr_trial_count = 0
            unit = unit_tbl[unit_tbl['unit_id'] == unit_id]
            unit_drift = load_drift(session, unit_id, data_type=data_type)
            
            if session_df is not None:
                go_cue_times = session_df['goCue_start_time']
                if unit_drift is not None:
                    if unit_drift['ephys_cut'][0] is not None:
                        if unit_drift['ephys_cut'][0] > pre_end - 2*60:
                            curr_pre_opto = False 
                        go_cue_times = go_cue_times[go_cue_times >= unit_drift['ephys_cut'][0]]         
                    if unit_drift['ephys_cut'][1] is not None:
                        if unit_drift['ephys_cut'][1] < post_start + 2*60:
                            curr_post_opto = False
                        if unit_drift['ephys_cut'][1] < post_end + 2*60:
                            curr_anti_opto = False
                        go_cue_times = go_cue_times[go_cue_times <= unit_drift['ephys_cut'][1]]
                if len(go_cue_times) < 100: 
                    curr_in_beh = False
                curr_trial_count = len(go_cue_times)
            opto_tagging_pre.append(curr_pre_opto)
            opto_tagging_post.append(curr_post_opto)
            anti.append(curr_anti_opto)
            trial_count.append(curr_trial_count)
            in_behavior.append(curr_in_beh)
        probes.extend([df[df['session_id']==session]['probe'].values[0]] * len(unit_tbl))
        rec_side.extend([df[df['session_id']==session]['side'].values[0]]*len(unit_tbl))  # recording side
        in_df.extend([beh] * len(unit_tbl))  # store session df for each unit


Processing ecephys_713854_2024-03-05_12-01-40
Processing ecephys_713854_2024-03-05_13-01-09
Processing ecephys_713854_2024-03-05_13-31-20
Processing ecephys_717120_2024-03-06_12-54-27
Processing ecephys_717120_2024-03-07_12-12-02
Processing ecephys_713854_2024-03-08_15-43-01
Processing ecephys_684930_2023-09-27_10-04-04
Processing ecephys_684930_2023-09-28_11-45-27
Processing ecephys_684930_2023-09-28_12-44-15
Processing ecephys_687697_2023-09-15_11-30-06
Processing ecephys_687697_2023-09-15_12-36-06
Processing ecephys_691893_2023-10-05_12-46-57
Processing ecephys_691893_2023-10-06_13-48-18
Processing behavior_716325_2024-05-31_10-31-14
Processing behavior_717121_2024-06-15_10-00-58
Processing behavior_751004_2024-12-19_11-50-37
Processing behavior_751004_2024-12-20_13-26-11
Processing behavior_751004_2024-12-21_13-28-28
Processing behavior_751004_2024-12-22_13-09-17
Processing behavior_751004_2024-12-23_14-20-03
Processing behavior_751769_2025-01-16_11-32-05
Processing behavior_751769

In [6]:
combined_tagged_units = pd.DataFrame({'session': all_sessions,
                                        'unit': all_units,
                                        'qc_pass': all_qc_pass_units,
                                        'opto_tagged': all_tagged_units,
                                        'opto_tagging_pre': opto_tagging_pre,
                                        'opto_tagging_post': opto_tagging_post,
                                        'anti': anti,
                                        'in_df': in_behavior,
                                        'trial_count': trial_count,
                                        'p_max': all_p_max,
                                        'sig_counts': all_sig_counts,
                                        'p_mean': all_p_mean,
                                        'lat_max_p': all_lat_max_p,
                                        'isi_violations': isi_v,
                                        'snr': snr,
                                        'eu': all_eu,
                                        'corr': all_corr,
                                        'amp': all_amp,
                                        'amp_raw': all_amp_raw,
                                        'peak': all_peak,
                                        'peak_raw': all_peak_raw,
                                        'wf': all_wf,
                                        'wf_raw': all_wf_raw,
                                        'wf_aligned': all_wf_aligned,
                                        'wf_2d': all_wf_2d,
                                        'wf_2d_raw': all_wf_2d_raw,
                                        'probe': probes,
                                        'y_loc': y_loc, 
                                        'rec_side': rec_side,
                                        'top': top,
                                        'bottom': bottom,
                                        'tag_loc': all_tag_loc,
                                        'fr': all_fr,
                                        'decoder': all_decoder,
                                        # 'peak_channel_ind': all_peak_channel_ind,
                                        'all_p_max': resp_p_all_conditions,
                                        'all_p_mean': mean_p_all_conditions,
                                        'all_lat_max_p': resp_lat_all_conditions,
                                        'all_corr': corr_all_conditions,
                                        'all_eu': eu_all_conditions,
                                        'all_sig_counts': sig_counts_all_conditions,
                                        'x_ccf': all_x,
                                        'y_ccf': all_y,
                                        'z_ccf': all_z
                                        }
)


In [7]:
# save dataframe in combined folder
with open(os.path.join('/root/capsule/scratch/combined/combine_unit_tbl', 'combined_unit_tbl.pkl'), 'wb') as f:
    pickle.dump(combined_tagged_units, f)

In [8]:
combined_tagged_units

,session,unit,qc_pass,opto_tagged,opto_tagging_pre,opto_tagging_post,anti,in_df,trial_count,p_max,...,decoder,all_p_max,all_p_mean,all_lat_max_p,all_corr,all_eu,all_sig_counts,x_ccf,y_ccf,z_ccf
0,ecephys_713854_2024-03-05_12-01-40,0,True,False,True,True,True,False,0,0.067565,...,sua,"[0.0675649587992235, 0.0675649587992235, 0.067...","[0.01756495879922349, 0.01756495879922349, -0....","[0.012637691061570422, 0.0154499707284117, 0.0...","[0.9816956669473178, 0.9816956669473178, 0.956...","[0.1806716624389166, 0.1806716624389166, 0.254...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",NaN,NaN,NaN
1,ecephys_713854_2024-03-05_12-01-40,1,True,False,True,True,True,False,0,0.133974,...,sua,"[0.13397357711152094, 0.13397357711152094]","[0.043973577111520945, 0.043973577111520945]","[0.013369887341150388, 0.013190711509878383]","[0.9736031429341969, 0.9736031429341969]","[0.19166737649409102, 0.19166737649409102]","[0.0, 0.0]",NaN,NaN,NaN
2,ecephys_713854_2024-03-05_12-01-40,2,True,False,True,True,True,False,0,-0.095125,...,sua,"[-0.09512476061606812, -0.09512476061606812]","[-0.1851247606160681, -0.2451247606160681]","[0.013687886667912607, 0.01320270290323151]","[0.9969125733013108, 0.9981435437578605]","[0.0800441711623354, 0.05573205958218562]","[0.0, 2.0]",NaN,NaN,NaN
3,ecephys_713854_2024-03-05_12-01-40,3,True,False,True,True,True,False,0,0.030265,...,sua,[0.030264686861479873],[-0.10973531313852014],[0.014005662938672003],[0.9927326793415915],[0.0950639340772765],[0.0],NaN,NaN,NaN
4,ecephys_713854_2024-03-05_12-01-40,4,True,False,True,True,True,False,0,0.113496,...,sua,[0.11349601093379369],[-0.13650398906620625],[0.015920421641616094],[0.9856689151944747],[0.14303780708431182],[1.0],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17843,behavior_ZS062_2021-05-09_17-15-10,TT7_SS_01,True,True,False,True,False,True,310,0.985263,...,sua,"[0.9852625709399964, 0.8852625709399964, 0.685...","[0.6452625709399964, 0.6452625709399964, 0.645...","[0.005856949951171875, 0.008878083279079862, 0...","[0.9972601149568114, 0.9972601149568114, 0.997...","[0.05340100078625947, 0.05340100078625947, 0.0...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",NaN,NaN,NaN
17844,behavior_ZS062_2021-05-10_19-15-51,TT2_SS_01,True,True,False,True,False,True,385,0.921359,...,sua,"[0.7213587114767099, 0.9213587114767099, 0.921...","[0.90135871147671, 0.90135871147671, 0.9013587...","[0.00536125, 0.004966449804687499, 0.005512949...","[0.9989019745576428, 0.9989019745576428, 0.998...","[0.19914732367639545, 0.19914732367639545, 0.1...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",NaN,NaN,NaN
17845,behavior_ZS062_2021-05-10_19-15-51,TT6_SS_03,True,True,False,True,False,True,385,0.961563,...,sua,"[0.9615634962842293, 0.7615634962842294, 0.861...","[0.9015634962842294, 0.9015634962842294, 0.901...","[0.0053947499511718755, 0.007493749755859375, ...","[0.9511787564847621, 0.9511787564847621, 0.951...","[0.12789753745452687, 0.12789753745452687, 0.1...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",NaN,NaN,NaN
17846,behavior_ZS062_2021-05-10_19-15-51,TT7_SS_01,True,True,False,True,False,True,385,0.870770,...,sua,"[0.8707695201516766, 0.7707695201516767, 0.670...","[0.5307695201516766, 0.5307695201516766, 0.530...","[0.007619888617621528, 0.013399374816894532, 0...","[0.9908667703752849, 0.9908667703752849, 0.990...","[0.07059006096124042, 0.07059006096124042, 0.0...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",NaN,NaN,NaN


In [9]:
unit_tbl = get_unit_tbl('behavior_754897_2025-03-13_11-20-42', 'curated')

In [10]:
unit_tbl.columns

Index(['unit_id', 'bl_max_p', 'p_max', 'p_mean', 'lat_max_p', 'lat_mean',
       'euc_max_p', 'corr_max_p', 'opto_pass', 'amp', 'peak', 'real_unit',
       'y_loc', 'pass_count', 'spike_times', 'ks_unit_id',
       'isi_violations_ratio', 'firing_rate', 'presence_ratio',
       'amplitude_cutoff', 'decoder_label', 'depth', 'snr', 'waveform_mean',
       'waveform_sd', 'default_qc', 'peak_wf', 'peak_wf_aligned', 'wf_2d',
       'LC_range_top', 'LC_range_bottom', 'tagged_loc', 'tagged',
       'amplitude_opt', 'peak_wf_opt', 'mat_wf_opt', 'peak_wf_opt_aligned',
       'mat_wf_raw', 'mat_wf_raw_aligned', 'peak_waveform_raw',
       'peak_waveform_raw_aligned', 'amplitude_raw', 'peak_raw',
       'mat_wf_raw_fake', 'mat_wf_raw_fake_aligned', 'peak_waveform_raw_fake',
       'peak_waveform_raw_fake_aligned', 'amplitude_raw_fake', 'peak_raw_fake',
       'loc_along_probe_x', 'x_ccf', 'y_ccf', 'z_ccf', 'loc_along_probe_y'],
      dtype='object')